# Parker Solar Probe Data Download Example

Tamar Ervin

- Downloading [PSP](https://link.springer.com/article/10.1007/s11214-015-0211-6) data with [PySPEDAS](https://pyspedas.readthedocs.io/en/latest/)
- Using [pyTplot](https://pytplot.readthedocs.io/en/latest/) to plot the data


In [ ]:
### install required packages
!pip install pyspedas pytplot
!pip install sunpy astrospice astropy

### upgrade the pyspedas package which pulls the data, has issues sometimes!
!pip install pyspedas --upgrade


In [ ]:
### don't worry if you get error messages!!!
import pyspedas
from pyspedas import time_string
from pytplot import tplot, get_data
import astrospice
import sunpy 
import sunpy.coordinates as scoords
import sys, os
import datetime
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt

## Download and Plot the Data

Example data to download:
- [FIELDS](https://link.springer.com/article/10.1007/s11214-016-0244-5): Radial, Tangential, Normal (RTN) magnetic field data
- [SWEAP/SPAN-I](https://iopscience.iop.org/article/10.3847/1538-4357/ac93f5) Proton: Radial, Tangential, Normal (RTN) proton velocity and proton density data
- [SWEAP/SPAN-I](https://iopscience.iop.org/article/10.3847/1538-4357/ac93f5) Alpha Particle: Radial, Tangential, Normal (RTN) alpha particle velocity and density

Data will download to a folder titled 'psp_data' in this same repo!

Don't worry if this takes a while to run! The data is at a very high cadence and takes a bit to download depending on the time range

## Plot the data using pyTplot

[pyTplot](https://pytplot.readthedocs.io/en/latest/) is a Python package that works with PySPEDAS to plot space physics data! It already has all the information needed to plot observables in terms of their units! You can also create your own plots of the data using matplotlib.


In [ ]:
### ------- TIME PERIOD OF INTEREST ------- ###
# this is an example of data from the heliospheric current sheet (HCS) crossing during PSP Encounter 15
time_range = ['2023-03-17/12:00', '2023-03-18/12:00']

Here we will access data from the electromagnetic fields instrument. We are interested in the radial (Br), tangential (Bt), and normal (Bn) components of the magnetic field!

In [ ]:
### ------- FIELDS: MAG RTN DATA ------- ###
fields_vars = pyspedas.psp.fields(trange=time_range, datatype='mag_RTN_4_Sa_per_Cyc')

### print out the variables stored in the magnetic field data
print(fields_vars)

### get the RTN magnetic field
B_RTN = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc')

### plot the data!
tplot(['psp_fld_l2_mag_RTN_4_Sa_per_Cyc'])

### CONVERT TIME FROM JULIAN TIME TO DATETIME OBJECT
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in B_RTN.times]

### CREATE DATAFRAME
rd = {'Time': date_obj, 'Br': B_RTN.y[:, 0], 'Bt': B_RTN.y[:, 1], 'Bn': B_RTN.y[:, 2]}
fields = pd.DataFrame(data=rd)

### SAVE DATAFRAME AS CSV
fields.to_csv(os.path.join('fields.csv')) 

Here we will access data from the proton instrument. We are interested in the radial (Br), tangential (Bt), and normal (Bn) components of the velocity. Tp is the proton temperature. Np is the proton number density.

In [ ]:
### ------- SPAN-I: PROTON (HYDROGEN) MOMENTS ------- ###
### download proton data
proton_vars = pyspedas.psp.spi(trange=time_range, datatype='sf00_l3_mom', level='l3')

### print out the variables stored in the proton data
print(proton_vars)

### get the RTN velocity, density, and temperature
Np = get_data('psp_spi_DENS')
vp_RTN = get_data('psp_spi_VEL_RTN_SUN')
Tp = get_data('psp_spi_TEMP')

### CONVERT TIME FROM JULIAN TIME TO DATETIME OBJECT
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in Np.times]

### CREATE DATAFRAME
rd = {'Time': date_obj, 'vr': vp_RTN.y[:, 0], 'vt': vp_RTN.y[:, 1], 'vn': vp_RTN.y[:, 2], 'Np': Np.y, 'Tp': Tp.y}
protons = pd.DataFrame(data=rd)

### SAVE DATAFRAME AS CSV
protons.to_csv(os.path.join('protons.csv'))

### plot the data!
tplot(['psp_spi_DENS', 'psp_spi_VEL_RTN_SUN', 'psp_spi_TEMP'])

Here we will access data from the alpha instrument. We are interested in the radial (Bra), tangential (Bta), and normal (Bna) components of the velocity of the alpha (Helium) particles. Ta is the alpha temperature. Na is the alpha number density.

In [ ]:
### ------- SPAN-I: ALPHA PARTICLE (HELIUM) MOMENTS ------- ###
### download alpha particle data
alpha_vars = pyspedas.psp.spi(trange=time_range, datatype='sf0a_l3_mom', level='l3')

### print out the variables stored in the alpha particle data
print(alpha_vars)

### READ IN SWEAP VELOCITY (RTN), DENSITY, AND TEMPERATURE DATA
Na = get_data('psp_spi_DENS')
va_RTN = get_data('psp_spi_VEL_RTN_SUN')
Ta = get_data('psp_spi_TEMP')

### CONVERT TIME FROM JULIAN TIME TO DATETIME OBJECT
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in Na.times]

### CREATE DATAFRAME
rd = {'Time': date_obj, 'vra': va_RTN.y[:, 0], 'vta': va_RTN.y[:, 1], 'vna': va_RTN.y[:, 2], 'Na': Na.y, 'Ta': Ta.y}
alphas = pd.DataFrame(data=rd)

### SAVE DATAFRAME AS CSV
alphas.to_csv(os.path.join('alphas.csv'))

### plot the data!
tplot(['psp_spi_DENS', 'psp_spi_VEL_RTN_SUN', 'psp_spi_TEMP'])

# Create Parker FIELDS/SWEAP Dataframe

Now that we have a dataframe for each instrument, we then use pandas to merge the dataframes as a function of time.



In [ ]:
### merge the PAS and MAG dataframes
merged_df = pd.merge_asof(fields, protons, on='Time', direction='backward')

### merge the HIS and newly merged dataframe
merged_df = pd.merge_asof(alphas, merged_df, on='Time', direction='backward')
merged_df = merged_df.set_index('Time')

In [ ]:
### look at your dataframe
merged_df

## Find the spacecraft trajectory

Now we will use astrospice to generate the trajectory of the spacecraft. This package includes a "kernel" called PSP which holds the coordinate positions of Parker Solar Probe throughout its lifetime. Then we generate the coordinates for the specific time frame we want (pulled from our dataframe) based on this kernel.

In [ ]:
### access the PSP kernel
kernels = astrospice.registry.get_kernels('psp','predict')

### Create SkyCoord for Parker in the inertial (J2000) frame
psp_inertial = astrospice.generate_coords(
    'SOLAR PROBE PLUS', pd.to_datetime(merged_df.index.to_list())
)

### IGNORE THE WARNINGS !!! DONT WORRY

In [ ]:
### read out these coordinates to see the structure
print(psp_inertial)

In [ ]:
### we transform from inertial coordinates (celestial coordinates) to heliocentric inertial coordinates (sun centered)
psp_inertial = psp_inertial.transform_to(scoords.HeliocentricInertial)

### We will now convert these to spherical coordinates (longitude, lattiude, radius)
psp_inertial.representation_type='spherical'

### print this out to see the difference
print(psp_inertial)

We can create a figure of the spacecraft coordinates in the inertial coordinate frame, and then you can look at it in Carrington coordinates! Feel free to experiment with colors: https://matplotlib.org/stable/gallery/color/named_colors.html

Some important things to remember about plots:
- axes labels!!, large fontsize is important!
- visible/readable colors (consider color blindless)
- big enough figures

In [ ]:
### setting up some figure parameters
plt.rcParams['font.family'] = 'Times New Roman' ## update the font to times new roman
plt.rcParams['font.size'] = 20 ## change the fontsize!

In [ ]:
### we will create a figure looking at the coordinates in various combinations
fig, axs = plt.subplots(1, 2, figsize=(16, 6)) ### this will allow us to create two subplots that are next to each other!!
ax1 = axs[0] ## this is the left axes
ax2 = axs[1] ## this is the right axes

### plot the longitude and latitude coordinates in the inertial frame
ax1.plot(psp_inertial.lon, psp_inertial.lat, color='lavender', linewidth=5)
# add axes labels
ax1.set(xlabel='Longitude [deg]', ylabel='Latitude [deg]')

### plot the longitude and radial coordinates in the inertial frame
ax2.plot(psp_inertial.lon, psp_inertial.distance, color='lightpink', linewidth=5)
# add axes labels
ax2.set(xlabel='Longitude [deg]', ylabel='Radial Distance [km]')

### save the figure!
plt.savefig(os.path.realpath('inertial_trajectory.png'))

Now we will look at the coordinate system in Carrington coordinates. These are coordinates that co-rotate with the Sun!

In [ ]:
### Transform to solar co-rotating frame (Carrington coordinates)
psp_carrington = psp_inertial.transform_to(
    scoords.HeliographicCarrington(observer="self")
)

Recreate the same figure as above but for the Carrington frame. Make sure to change the naming of the figure to make sense!

In [ ]:
### create this figure for the Carrington frame


Now we will add the position information to our merged dataframe and save as a CSV file!

In [ ]:
### ADD POSITION INFORMAITON AND SAVE
parker = merged_df.copy()
parker['lon'] = psp_carrington.lon.value ## longitude in carrington coordinates
parker['lat'] = psp_carrington.lat.value ## latitude in carrington coordinates
parker['rAU'] = psp_carrington.radius.to(u.AU).value ## radial distance from the Sun in AU (1AU is the Earth-Sun distance)
parker.to_csv(os.path.join('parker.csv'))

In [ ]:
### look at the dataframe
parker

# Radial Scaling of the Magnetic Field

We are interested in understanding how the radial magnetic field (Br) scales with radial distance. Make a figure using the data stored in your dataframe to look at the radial scaling of the field. Make sure to think about the plotting "best practices" described above!

You should plot data directly from the dataframe, do some googling to figure out how to do this!

Ideas:
- create a figure looking at radial distance versus radial (Br) magnetic field strength
- look at the radial scaling of the magnetic field magnitude (|B|) calculated from Br, Bt, Bn
- look at radial scaling of the proton density
- calculate the proton and alpha pressure (think PV=nRT) and look at its radial scaling

You can fit the scaling of these parameters as a function of distance to understand how they change!

In [ ]:
### read in the dataframe
parker = pd.read_csv(os.path.join('parker.csv'))

### convert the time variable from a string to a datetime object (this allows you to make plots!)
parker['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in parker.Time]

### look at the dataframe
parker
